In [39]:
import pickle as pk

In [35]:
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile, mutual_info_regression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
# VIEWING OPTIONS IN THE NOTEBOOK
from sklearn import set_config; set_config(display='diagram')

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns

In [14]:
df = pd.read_pickle('attackers_df.pkl')

In [29]:
df.shape

(2085, 29)

In [16]:
numerical_features = [ 'games_2022', 'minutes_played_2022', 'goals_2022',
       'assists_2022', 'goals_against_2022', 'goals_for_2022',
       'clean_sheet_2022', 'height_in_cm', 'age', 
       'club_value', 'squad_size', 'term_days_remaining',
       'value_goals_for_2022', 'yellow_cards_2022', 'red_cards_2022']
categorical_features = ['sub_position', 'foot']
ordinal_features = ['country_of_citizenship','current_club_domestic_competition_id', 'current_club_name']
target = 'market_value_in_eur'

In [31]:
len(numerical_features)+len(categorical_features)+len(ordinal_features)

20

In [50]:
df.sub_position.unique()

array(['Centre-Forward', 'Right Winger', 'Attacking Midfield',
       'Left Winger', 'Second Striker', nan], dtype=object)

In [17]:
numerical_transformer =  MinMaxScaler()


In [18]:
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

In [19]:
feat_ordinal_dict = {
    "country_of_citizenship": ['Indonesia', 'Tajikistan', 'Faroe Islands', 'Moldova', 'Belarus', 'Montserrat', 'Uganda', 'Northern Ireland', 'Peru', 'Azerbaijan', 'Curacao', 'Costa Rica', 'Ireland', 'Iceland', 'Mozambique', 'Congo', 'Sierra Leone', 'Grenada', 'Finland', 'Panama', 'Guadeloupe', 'South Africa', 'Iraq', 'Chad', 'Tunisia', 'Angola', 'Venezuela', 'Greece', 'Australia', 'North Macedonia', 'Tanzania', 'Cape Verde', 'Slovakia', 'Scotland', 'Palestine', 'Luxembourg', 'Ukraine', 'Latvia', 'Jordan', 'Martinique', 'Comoros', 'Russia', 'Romania', 'Honduras', 'Turkey', 'Montenegro', 'Zimbabwe', 'Ecuador', 'Chile', 'Bosnia-Herzegovina', 'The Gambia', 'Denmark', 'Dominican Republic', 'Kosovo', 'Mali', 'Guinea', 'Mauritania', 'New Zealand', 'Guinea-Bissau', 'Uzbekistan', 'Armenia', 'Netherlands', 'Japan', 'Morocco', 'Gabon', 'Ghana', 'Slovenia', 'Togo', 'Philippines', 'Israel', 'Switzerland', 'Trinidad and Tobago', 'Kazakhstan', 'Austria', 'Suriname', 'Cameroon', 'Burkina Faso', 'Algeria', 'Paraguay', 'Belgium', 'Iran', 'Benin', 'Senegal', 'DR Congo', 'Sweden', 'Serbia', 'Wales', 'Croatia', 'Jamaica', 'Spain', 'Albania', 'Nigeria', 'Zambia', 'Portugal', 'Uruguay', 'Italy', 'United States', "Cote d'Ivoire", 'Germany', 'Colombia', 'Poland', 'Argentina', 'France', 'Canada', 'Brazil', 'Norway', 'Mexico', 'Hungary', 'Georgia', 'Czech Republic', 'Korea, South', 'England', 'Egypt'],
    "current_club_name":['Fk Minaj', 'Pfk Lviv', 'Gaziantep Fk', 'Ingulets Petrove', 'Kryvbas Kryvyi Rig', 'Metal Kharkiv', 'Nk Veres Rivne', 'Metalist 1925 Kharkiv', 'Chornomorets Odessa', 'Motherwell Fc', 'Dundee United Fc', 'Pas Lamia 1964', 'Pas Giannina', 'Lyngby Bk', 'Kilmarnock Fc', 'Rukh Lviv', 'Fk Oleksandriya', 'Ac Horsens', 'Kolos Kovalivka', 'Asteras Tripolis', 'Aberdeen Fc', 'Ross County Fc', 'St Mirren Fc', 'Go Ahead Eagles Deventer', 'St Johnstone Fc', 'Livingston Fc', 'Viborg Ff', 'Fakel Voronezh', 'Fc Volendam', 'Volos Nps', 'Ionikos Nikeas', 'Ofi Kreta', 'Torpedo Moskau', 'Istanbulspor', 'Fc Emmen', 'Apo Levadiakos', 'Panetolikos Gfs', 'Odense Boldklub', 'Rfc Seraing', 'Zorya Lugansk', 'Portimonense Sc', 'Sbv Excelsior Rotterdam', 'Gd Chaves', 'Heart Of Midlothian Fc', 'Vorskla Poltava', 'Aarhus Gf', 'Rio Ave Fc', 'Sv Zulte Waregem', 'Vv St Truiden', 'Fk Nizhny Novgorod', 'Umraniyespor', 'Sparta Rotterdam', 'Aalborg Bk', 'Boavista Porto Fc', 'Silkeborg If', 'Randers Fc', 'Kv Oostende', 'Ural Ekaterinburg', 'Fc Pacos De Ferreira', 'Fc Vizela', 'Casa Pia Ac', 'Rkc Waalwijk', 'Kas Eupen', 'Fc Arouca', 'Sc Cambuur Leeuwarden', 'Nec Nijmegen', 'Atromitos Athen', 'Fk Orenburg', 'Fk Sochi', 'Kv Kortrijk', 'Cd Santa Clara', 'Cs Maritimo', 'Fk Khimki', 'Hibernian Fc', 'Ac Ajaccio', 'Gd Estoril Praia', 'Fc Famalicao', 'Fortuna Sittard', 'Kvc Westerlo', 'Akhmat Grozny', 'Giresunspor', 'Mke Ankaragucu', 'Kv Mechelen', 'Kayserispor', 'Brondby If', 'Vitoria Guimaraes Sc', 'Sivasspor', 'Fc Twente Enschede', 'Fatih Karagumruk', 'Vitesse Arnheim', 'Sc Heerenveen', 'Cercle Brugge', 'Hatayspor', 'Dynamo Kiew', 'Alanyaspor', 'Kasimpasa', 'Krylya Sovetov Samara', 'Fc Midtjylland', 'Aj Auxerre', 'Aris Thessaloniki', 'Fk Rostov', 'Fc Nordsjaelland', 'Gil Vicente Fc', 'Fc Utrecht', 'Adana Demirspor', 'Antalyaspor', 'Aek Athen', 'Oud Heverlee Leuven', 'Fc Schalke 04', 'Konyaspor', 'Paok Thessaloniki', 'Clermont Foot 63', 'Sco Angers', 'Standard Luttich', 'Fc Groningen', 'Vfl Bochum', 'Rsc Charleroi', 'Shakhtar Donetsk', 'Istanbul Basaksehir Fk', 'Sampdoria Genua', 'Hellas Verona', 'Fc Cadiz', 'Us Cremonese', '1 Fc Koln', 'Fc Toulouse', 'Sk Dnipro 1', 'Panathinaikos Athen', 'Royale Union Saint Gilloise', 'Kaa Gent', 'Krc Genk', 'Rc Strassburg Alsace', 'Fk Krasnodar', 'Stade Brest 29', 'Spezia Calcio', 'Fc Elche', 'Us Lecce', 'Vfb Stuttgart', 'Fc Empoli', 'Fc Kopenhagen', 'Lokomotiv Moskau', 'Hertha Bsc', 'Olympiakos Piraus', 'Real Valladolid', 'Es Troyes Ac', 'Royal Antwerpen Fc', 'Sv Werder Bremen', 'Fc Augsburg', 'Az Alkmaar', 'Rcd Mallorca', 'Stade Reims', 'Zska Moskau', 'Rsc Anderlecht', 'Spartak Moskau', 'Fc Lorient', '1 Fsv Mainz 05', 'Rayo Vallecano', 'Ud Almeria', 'Espanyol Barcelona', 'Ac Monza', '1 Fc Union Berlin', 'Feyenoord Rotterdam', 'Dinamo Moskau', 'Glasgow Rangers', 'Celtic Glasgow', 'Fc Girona', 'Montpellier Hsc', 'Brighton Amp Hove Albion', 'Rc Lens', 'Ca Osasuna', 'Celta Vigo', 'Fc Nantes', 'Udinese Calcio', 'Fc Bologna', 'Sc Braga', 'Trabzonspor', 'Us Salernitana 1919', 'Besiktas Istanbul', 'Tsg 1899 Hoffenheim', 'Zenit St Petersburg', 'Psv Eindhoven', 'Fenerbahce Istanbul', 'Fc Getafe', 'Fc Turin', 'Vfl Wolfsburg', 'Borussia Monchengladbach', 'Sc Freiburg', 'Fc Brugge', 'Fc Valencia', 'Galatasaray Istanbul', 'Olympique Lyon', 'Afc Bournemouth', 'Us Sassuolo', 'Lazio Rom', 'Sporting Lissabon', 'Nottingham Forest', 'Fc Porto', 'Olympique Marseille', 'Fc Sevilla', 'Ac Florenz', 'Ogc Nizza', 'Fc Fulham', 'As Monaco', 'Fc Southampton', 'Athletic Bilbao', 'Ajax Amsterdam', 'Losc Lille', 'Real Betis Sevilla', 'Leeds United', 'Real Sociedad San Sebastian', 'Atalanta Bergamo', 'Wolverhampton Wanderers', 'Fc Everton', 'Eintracht Frankfurt', 'Benfica Lissabon', 'Ac Mailand', 'As Rom', 'Fc Brentford', 'Crystal Palace', 'Borussia Dortmund', 'Fc Stade Rennes', 'Fc Villarreal', 'Juventus Turin', 'Newcastle United', 'Aston Villa', 'West Ham United', 'Leicester City', 'Bayer 04 Leverkusen', 'Rasenballsport Leipzig', 'Atletico Madrid', 'Ssc Neapel', 'Inter Mailand', 'Manchester United', 'Fc Chelsea', 'Real Madrid', 'Fc Bayern Munchen', 'Fc Arsenal', 'Fc Barcelona', 'Tottenham Hotspur', 'Fc Liverpool', 'Fc Paris Saint Germain', 'Manchester City'],
    "current_club_domestic_competition_id": ['UKR1', 'DK1', 'GR1', 'SC1', 'BE1', 'RU1', 'NL1', 'TR1', 'PO1', 'FR1', 'L1', 'IT1', 'ES1', 'GB1'],
}
    
feat_ordinal = sorted(feat_ordinal_dict.keys())
feat_ordinal_values_sorted = [feat_ordinal_dict[i] for i in feat_ordinal]
    
ordinal_transformer = OrdinalEncoder(
    categories=feat_ordinal_values_sorted,
 #   handle_unknown="use_encoded_value",
  #  unknown_value=-1  # Considers unknown values as worse than "missing"
)




In [20]:
transformer = ColumnTransformer([
    ('num_tr', numerical_transformer, numerical_features),
    ('cat_tr', categorical_transformer, categorical_features),
    ('ord_tr', ordinal_transformer, ordinal_features)

])

In [21]:
transformer.fit(df)

ColumnTransformer(transformers=[('num_tr', MinMaxScaler(),
                                 ['games_2022', 'minutes_played_2022',
                                  'goals_2022', 'assists_2022',
                                  'goals_against_2022', 'goals_for_2022',
                                  'clean_sheet_2022', 'height_in_cm', 'age',
                                  'club_value', 'squad_size',
                                  'term_days_remaining', 'value_goals_for_2022',
                                  'yellow_cards_2022', 'red_cards_2022']),
                                ('cat_tr',
                                 OneHotEncoder(handle_unknown='i...
                                                             'Pas Giannina',
                                                             'Lyngby Bk',
                                                             'Kilmarnock Fc',
                                                             'Rukh Lviv',
                                                             'Fk Oleksandriya',
                                                             'Ac Horsens',
                                                             'Kolos Kovalivka',
                                                             'Asteras Tripolis',
                                                             'Aberdeen Fc',
                                                             'Ross County Fc',
                                                             'St Mirren Fc',
                                                             'Go Ahead Eagles '
                                                             'Deventer',
                                                             'St Johnstone Fc',
                                                             'Livingston Fc',
                                                             'Viborg Ff',
                                                             'Fakel Voronezh',
                                                             'Fc Volendam',
                                                             'Volos Nps', ...]]),
                                 ['country_of_citizenship',
                                  'current_club_domestic_competition_id',
                                  'current_club_name'])])

In [24]:
transformed_df = transformer.transform(df)

In [27]:
transformed_df.shape

(2085, 28)

In [32]:
pd.DataFrame(
    transformed_df, 
    columns=transformer.get_feature_names_out()
).head()

,num_tr__games_2022,num_tr__minutes_played_2022,num_tr__goals_2022,num_tr__assists_2022,num_tr__goals_against_2022,num_tr__goals_for_2022,num_tr__clean_sheet_2022,num_tr__height_in_cm,num_tr__age,num_tr__club_value,...,cat_tr__sub_position_Right Winger,cat_tr__sub_position_Second Striker,cat_tr__sub_position_nan,cat_tr__foot_Both,cat_tr__foot_Left,cat_tr__foot_Right,cat_tr__foot_nan,ord_tr__country_of_citizenship,ord_tr__current_club_domestic_competition_id,ord_tr__current_club_name
0,0.44,0.175263,0.000000,0.000000,0.490196,0.079365,0.000000,0.930693,0.12,0.090909,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,41.0,5.0,27.0
1,0.56,0.561127,0.037037,0.066667,0.274510,0.269841,0.416667,0.866337,0.80,0.272727,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,62.0,4.0,48.0
2,0.68,0.648042,0.037037,0.066667,0.568627,0.222222,0.250000,0.935644,0.64,0.090909,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,41.0,5.0,27.0
3,0.80,0.450812,0.111111,0.133333,0.725490,0.253968,0.250000,0.920792,0.40,0.090909,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,41.0,5.0,27.0
4,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.905941,0.52,0.272727,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,36.0,4.0,48.0


In [33]:
X, y = transformed_df, df[target]

In [36]:
gbr = GradientBoostingRegressor(random_state=0, verbose = 0)


In [37]:
results = cross_validate (gbr, X, y)

In [38]:
results

{'fit_time': array([0.45774174, 0.49865603, 0.46537495, 0.47438908, 0.44382882]),
 'score_time': array([0.00336814, 0.00190616, 0.00181198, 0.00208926, 0.00259995]),
 'test_score': array([0.78590607, 0.57591925, 0.74314818, 0.72069637, 0.72836504])}

In [47]:
mean_score = np.mean(results['test_score'])
mean_score

0.7108069826642252

In [40]:
gbr.fit(X, y)

GradientBoostingRegressor(random_state=0)

In [43]:
# save the model to disk
filename = 'gbr_1.sav'
pk.dump(gbr, open(filename, 'wb'))

In [44]:
# save the pipeline to disk
filename = 'transformer_1.sav'
pk.dump(transformer, open(filename, 'wb'))

We need 20 inputs --> preproc takes care

In [54]:
player = pd.DataFrame()

In [ ]:
"""numerical_features = [ 'games_2022', 'minutes_played_2022', 'goals_2022',
       'assists_2022', 'goals_against_2022', 'goals_for_2022',
       'clean_sheet_2022', 'height_in_cm', 'age', 
       'club_value', 'squad_size', 'term_days_remaining',
       'value_goals_for_2022', 'yellow_cards_2022', 'red_cards_2022']
categorical_features = ['sub_position', 'foot']
ordinal_features = ['country_of_citizenship','current_club_domestic_competition_id', 'current_club_name']
target = 'market_value_in_eur'"""

In [55]:
player['games_2022'] =[20]
player['minutes_played_2022'] =[1800] 
player['goals_2022'] =[200]
player['assists_2022'] =[15] 
player['goals_against_2022'] =[20] 
player['goals_for_2022'] =[10]
player['clean_sheet_2022'] =[7] 
player['height_in_cm'] =[186] 
player['age'] =[31] 
player['club_value'] =[50000000] 
player['squad_size'] =[18] 
player['term_days_remaining'] =[200]
player['value_goals_for_2022'] =[50000000] 
player['yellow_cards_2022'] =[2] 
player['red_cards_2022'] =[0]
player['sub_position'] =['Left Winger'] 
player['foot'] =['Right']
player['country_of_citizenship'] =['Italy']
player['current_club_domestic_competition_id'] =['IT1'] 
player['current_club_name'] = ['Juventus Turin']

In [56]:
player

,games_2022,minutes_played_2022,goals_2022,assists_2022,goals_against_2022,goals_for_2022,clean_sheet_2022,height_in_cm,age,club_value,squad_size,term_days_remaining,value_goals_for_2022,yellow_cards_2022,red_cards_2022,sub_position,foot,country_of_citizenship,current_club_domestic_competition_id,current_club_name
0,20,1800,200,15,20,10,7,186,31,50000000,18,200,50000000,2,0,Left Winger,Right,Italy,IT1,Juventus Turin


In [59]:
player_transformed = transformer.transform(player)

In [60]:
my_value = gbr.predict(player_transformed)

In [61]:
my_value

array([80626632.50546265])